In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import re
import seaborn as sns
import sys
import urllib3
import json
import operator
import collections

import CPTAC.Endometrial as CPTAC

Welcome to the CPTAC data service package. Available datasets may be
viewed using CPTAC.list(). In order to access a specific data set,
import a CPTAC subfolder using either 'import CPTAC.Dataset' or 'from
CPTAC import Dataset'.
******
Version: 0.2.5
******
Loading Endometrial CPTAC data:
Loading Dictionary...
Loading Clinical Data...
Loading Acetylation Proteomics Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Mutation Data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but the data may not be in a
publication until July 1, 2019. Please see
https://proteomics.cancer.gov/data-portal/about/data-use-agreement or
enter embargo() to open the webpage for more details.


In [2]:
somatic = CPTAC.get_somatic()
proteomics = CPTAC.get_proteomics()
phos = CPTAC.get_phosphoproteomics()

## Step 1: Enter gene name
# INPUT: 

In [3]:
gene = "ARID1A" #Name of any gene of which you are interested in examing the effects of mutation

## Step 2: Select type of data
Set "data_type" to "proteomics" for proteomic abundance data <br><br>
Set "data_type" to "phosphoproteomics" for phosphoproteomic data <br><br>
Set "data_type" to "transcriptomics" for transcription data
# INPUT:

In [4]:
data_type = "proteomics" #Options are "proteomics", "phosphoproteomics", "transcriptomics"

## Step 3: Select scope of data analysis
Set "scope" to "cis" to examine cis effect of gene mutation on the protein it codes for <br> <br>
Set "scope" to "interacting" to examine the effect of gene mutation on known interacting proteins (interacting proteins are automatically generated through interacting with STRING database API). If using scope, set "number_of_interacting_proteins" to the number of interacting proteins you want to pull from the STRING database <br><br>
Set "scope" to "all" to look at the effect on all proteins in our dataset
# INPUT:

In [5]:
scope = "interacting" #Options are "cis", "interacting", "all"
number_of_interacting_proteins = 10 #Parameter only necessary if examining "interacting", else set to 0

## Step 4: Select Question

Set "question" to "mutated_vs_wildtype" to compare mutated and wildtype groups <br><br>
Set "question" to "hotspot_vs_wildtype" to compare individuals with a hotspot mutation with wildtype <br><br>
Set "question" to "truncation_vs_wildtype" to compare individuals with a truncating mutation with wildtype
# INPUT:

In [6]:
question = "mutated_vs_wildtype" #Options are .....

## Step 5: Set Dataframe to Selected Data

In [7]:
if data_type == "proteomics":
    dataframe = CPTAC.get_proteomics()
    
elif data_type == "phosphoproteomics":
    dataframe = CPTAC.get_phosphoproteomics()
    
elif data_type == "transcriptomics":
    dataframe = CPTAC.get_transcriptomics()
    
else:
    print("Invalid data_type value entered! See step 2 above.")

## Step 6: Generate Protein List Based on the Selected Scope

In [8]:
protein_list = []    

'''Just add the gene itself if cis'''
if scope == "cis":
    protein_list = [gene]
    
    '''If interacting, get interacting proteins from STRING database'''
elif scope == "interacting":
    
    '''Use urllib3 to access the string database api, gather list of interacting proteins'''
    urllib3.disable_warnings()
    string_api_url = "https://string-db.org/api"
    output_format = "json"
    method = "network"

    '''Use the specified gene and homo sapiens species code'''
    my_gene = [gene]
    species = "9606"

    '''Format the api request to collect the appropriate information'''
    request_url = string_api_url + "/" + output_format + "/" + method + "?"
    request_url += "identifiers=%s" % "%0d".join(my_gene)
    request_url += "&" + "species=" + species
    request_url += "&" + "limit=" + str(number_of_interacting_proteins)

    '''Send a request to the API, print the response status'''
    try:
        http = urllib3.PoolManager()
        response = http.request('GET',request_url)
        #print("Accessing String database, response status: ", response.status)
    except urllib3.HTTPError as err:
        error_message = err.read()
        print(error_message)
        sys.exit()

    interacting_proteins = []
    if response.status == 200: 
        '''Get the data from the API's response'''
        data = response.data
        y = json.loads(data)

        '''Make a list of the resulting interacting proteins'''
        for entry in y:
            if entry["preferredName_A"] not in interacting_proteins:
                interacting_proteins.append(entry["preferredName_A"])
            if entry["preferredName_B"] not in interacting_proteins:
                interacting_proteins.append(entry["preferredName_B"])
                
        for ip in interacting_proteins:
            protein_list.append(ip)
    else:
        print("\nSpecified gene was not found in String database, double check that you have it correctly!")
        protein_list.append(gene)

    '''If all, add all proteins in our dataset'''
elif scope == "all":
    protein_list = proteomics.columns

    '''If none of these, invalid'''
else:
    print("Invalid scope value entered! See step 3 above.")
    
'''Display list of proteins selected'''
print("Protein List: \n")
for pro in protein_list:
    print(pro)

Protein List: 

SMARCB1
SMARCC1
SMARCA2
SMARCC2
ARID1A
SMARCE1
PHF10
ARID1B
SMARCD1
SMARCD2
SMARCA4


## Step 7: Format Dataframe Based on Question

#### Question: Mutated vs. Wildtype

This will generate a dataframe with the gene mutation, sample status, and columns for either proteomics, phosphoproteomics, or transcriptomics for every protein (or every site in every protein, if phosphorylation).

In [9]:
if question == "mutated_vs_wildtype":
    '''Get dataframe for mutations, select sample status and gene mutation only'''
    genedf = CPTAC.compare_mutations(dataframe, protein_list[0], gene)
    genedf = genedf[[gene+"_Mutation", "Sample_Status"]]
    '''Loop through every protein in protein list, add in omics'''
    for protein in protein_list:
        protdf = CPTAC.compare_mutations(dataframe, protein, gene)
        if protdf is not None:
            for col in protdf.columns:
                if col not in genedf.columns:
                    genedf[col] = protdf[col]

genedf.head()

,ARID1A_Mutation,Sample_Status,SMARCB1_omics,SMARCC1_omics,SMARCA2_omics,SMARCC2_omics,ARID1A_omics,SMARCE1_omics,PHF10_omics,ARID1B_omics,SMARCD1_omics,SMARCD2_omics,SMARCA4_omics
S001,Missense_Mutation,Tumor,-0.111,0.0426,-0.9270,-0.451,-0.8790,-0.3950,0.0372,-0.0204,-0.328,-0.453,0.21700
S002,Nonsense_Mutation,Tumor,-0.639,0.0112,-1.1000,-0.627,-1.4100,-0.5560,0.0288,-0.3690,-0.418,-0.364,0.05670
S003,Wildtype_Tumor,Tumor,-0.352,-0.2320,-0.3560,-0.537,-1.0400,-0.5400,-0.2710,0.0225,-0.648,-0.409,-0.40200
S005,Wildtype_Tumor,Tumor,0.190,0.7800,-0.7170,-0.102,0.3850,0.0968,-0.1500,-0.1130,0.186,0.340,0.82900
S006,Wildtype_Tumor,Tumor,-0.829,-0.7890,0.0593,-0.432,0.0106,0.0897,-0.9080,-0.8190,-0.599,-0.186,-0.00505
